# 앙상블 검색기(Ensemble Retriever)

1. 여러검색기 통합
2. 결과 재순위화: `RRF(Reciprocal Rank Fusion)`알고리즘을 통해 순위를 조정 - 보통 `Ensemble Algorithmn`을 사용했다고 하기보단, `RRF`라고 말함
https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf
3. 하이브리드검색: 주로 `sparse retriever`(bm25)와 `dense retriver`(임베딩 유사도)를 결합하여 사용
4. 서로의 단점을 상호보완

**장점**
- `Sparse Retriever`: 키워드 기반 검색에 효과적
- `Dense Retriever`: 의지먹 유사성 기반 검색에 효과적

In [18]:
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# 샘플 문서 리스트
doc_list = [
    "I like apples",
    "I like apple company",
    "I like apple's iphone",
    "Apple is my favorite company",
    "I like apple's ipad",
    "I like apple's macbook",
]

In [20]:
bm25_retriever = BM25Retriever.from_texts(doc_list)
bm25_retriever.k = 1

In [21]:
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

chroma_vector_sotre=Chroma.from_texts(
    doc_list, embedding=embedding
)

chroma_retriever = chroma_vector_sotre.as_retriever(search_kwargs={"k":1})

In [22]:
ensemble_retriever = EnsembleRetriever(
    retrievers = [bm25_retriever, chroma_retriever],
    weights=[0.7, 0.3]
)

In [23]:
query = "Apple company makes my favorite iphone"
ensemble_result = ensemble_retriever.invoke(query)
bm25_result = bm25_retriever.invoke(query)
chroma_result = chroma_retriever.invoke(query)

In [24]:
# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]")
for doc in ensemble_result:
    print(f"Content: {doc.page_content}")
    print()

print("[BM25 Retriever]")
for doc in bm25_result:
    print(f"Content: {doc.page_content}")
    print()

print("[Chroma Retriever]")
for doc in chroma_result:
    print(f"Content: {doc.page_content}")
    print()

[Ensemble Retriever]
Content: Apple is my favorite company

[BM25 Retriever]
Content: Apple is my favorite company

[Chroma Retriever]
Content: Apple is my favorite company



# config설정

In [26]:
from langchain_core.runnables import ConfigurableField


ensemble_retriever = EnsembleRetriever(
    # 리트리버 목록을 설정합니다. 여기서는 bm25_retriever와 faiss_retriever를 사용합니다.
    retrievers=[bm25_retriever, chroma_retriever],
).configurable_fields(
    weights=ConfigurableField(
        # 검색 매개변수의 고유 식별자를 설정합니다.
        id="ensemble_weights",
        # 검색 매개변수의 이름을 설정합니다.
        name="Ensemble Weights",
        # 검색 매개변수에 대한 설명을 작성합니다.
        description="Ensemble Weights",
    )
)

In [27]:
#옵션의 가중치를 [1, 0]으로 설정하여 모든 검색 결과의 가중치가 BM25 retriever 에 더 많이 부여 되도록 합니다.
config = {"configurable": {"ensemble_weights": [1, 0]}}

# config 매개변수를 사용하여 검색 설정을 지정합니다.
docs = ensemble_retriever.invoke("my favorite fruit is apple", config=config)
docs  # 검색 결과인 docs를 출력합니다.

[Document(page_content='Apple is my favorite company'),
 Document(page_content='I like apples')]

In [28]:
#이번에는 검색시 모든 검색 결과의 가중치가 FAISS retriever 에 더 많이 부여 되도록 합니다.
config = {"configurable": {"ensemble_weights": [0, 1]}}

# config 매개변수를 사용하여 검색 설정을 지정합니다.
docs = ensemble_retriever.invoke("my favorite fruit is apple", config=config)
docs  # 검색 결과인 docs를 출력합니다.

[Document(page_content='I like apples'),
 Document(page_content='Apple is my favorite company')]